In [203]:
import pyreadr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import chi2_contingency

In [204]:
data_2014 = pyreadr.read_r('cleaned_data/data14.rds')
df_14=data_2014[None]

data_2015_18 = pyreadr.read_r('cleaned_data/data1518.rds')
df_1518=data_2015_18[None]

'''
with pd.ExcelWriter('output.xlsx') as writer:
    df.to_excel(writer, sheet_name='Sheet_name_1')
'''

"\nwith pd.ExcelWriter('output.xlsx') as writer:\n    df.to_excel(writer, sheet_name='Sheet_name_1')\n"

In [205]:
titles=df_14.keys()
titles2 = df_1518.keys()
print(titles)
print(titles2)

Index(['xstate', 'imonth', 'genhlth', 'physhlth', 'poorhlth', 'hlthpln1',
       'persdoc2', 'medcost', 'checkup1', 'exerany2', 'sex', 'educa',
       'employ1', 'income2', 'flushot6', 'howlong', 'lastpap2', 'lastsig3',
       'hadsigm3', 'xhcvu651', 'xtotinda', 'drvisits', 'medicare', 'hlthcvr1',
       'nocov121', 'medscost', 'carercvd', 'year'],
      dtype='object')
Index(['xstate', 'imonth', 'genhlth', 'physhlth', 'poorhlth', 'hlthpln1',
       'persdoc2', 'medcost', 'checkup1', 'exerany2', 'sex', 'educa',
       'employ1', 'income2', 'flushot6', 'howlong', 'lastpap2', 'lastsig3',
       'hadsigm3', 'xhcvu651', 'xtotinda', 'year', 'drvisits', 'medicare',
       'hlthcvr1', 'nocov121', 'medscost', 'carercvd'],
      dtype='object')


In [206]:
low_income14 = df_14.loc[df_14['income2'] <= 4]
low_income1518 = df_1518.loc[df_1518['income2'] <= 4]

In [207]:
#function to return the counts of people for each category in a given column
def getCounts(df, num_cols, threshold = 7):
    counts = [0 for i in range(num_cols)]
    for row in df:
        value = (int)(row)
        if value < threshold:
            counts[value-1] += 1
    return counts

In [208]:
#start of chi2 test for general health data
gen_health14 = df_14['genhlth']
gen_health14

0        1.0
1        3.0
2        2.0
3        1.0
4        3.0
        ... 
51853    3.0
51854    2.0
51855    2.0
51856    1.0
51857    2.0
Name: genhlth, Length: 51858, dtype: float64

In [209]:
gen_health1518 = df_1518['genhlth']
gen_health1518

0         3.0
1         3.0
2         3.0
3         1.0
4         3.0
         ... 
193644    3.0
193645    3.0
193646    1.0
193647    2.0
193648    1.0
Name: genhlth, Length: 193649, dtype: float64

In [210]:
#account for much larger data by taking random sample of same size of 2014 dataset
sample1518 = gen_health1518.sample(n = 51858, random_state = 1)
sample1518

120187    4.0
74314     2.0
10751     5.0
34734     2.0
83776     5.0
         ... 
56529     4.0
101909    3.0
36504     2.0
26896     2.0
126199    4.0
Name: genhlth, Length: 51858, dtype: float64

In [211]:
#get counts for each value of general health in 2014 data set
counts14 = getCounts(gen_health14, 5)
counts14

[10706, 18539, 14412, 5803, 2254]

In [212]:
#get counts for each value of general health in 2015-2018 data set sample
counts1518 = getCounts(sample1518,5)
counts1518

[10274, 18518, 14806, 5884, 2257]

In [213]:
#number of non refused or not sure answers in data set
total14 = np.sum(counts14)
total1518 = np.sum(counts1518)

print("2014 total", total14)
print("2015-2018 total",total1518)

2014 total 51714
2015-2018 total 51739


In [214]:
#create combined matrix
genhealth = pd.DataFrame([counts14, counts1518], index = ["2014", "2015-2018"], columns = ["Excellent", "Very Good", "Good", "Fair", "Poor"])
genhealth

,Excellent,Very Good,Good,Fair,Poor
2014,10706,18539,14412,5803,2254
2015-2018,10274,18518,14806,5884,2257


In [215]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2014', x = genhealth.columns, y = genhealth.iloc[0]),
                       go.Bar(name = '2015-18', x = genhealth.columns, y = genhealth.iloc[1])])
fig.update_layout(barmode='group')
fig.show()

In [216]:
#perform chi2 test
test = chi2_contingency(genhealth)
test

(14.77760328295258,
 0.005185422461637733,
 4,
 array([[10487.46503243, 18524.02248364, 14605.46965289,  5842.08788532,
          2254.95494572],
        [10492.53496757, 18532.97751636, 14612.53034711,  5844.91211468,
          2256.04505428]]))

In [217]:
#look at expected values of the data
#it appears that if anything the general health in 2014 is better than 2015-2018 (at least the sample I chose)
df = test[3]

ev = pd.DataFrame(data = df[:,:], index = ["2014", "2015-2018"],
                  columns = ["Excellent", "Very Good", "Good", "Fair", "Poor"]).round(2)

ev

,Excellent,Very Good,Good,Fair,Poor
2014,10487.47,18524.02,14605.47,5842.09,2254.95
2015-2018,10492.53,18532.98,14612.53,5844.91,2256.05


In [218]:
chi2 = test[0]
chi2

14.77760328295258

In [219]:
#note since we have so many data points the p value may be artifically lower
#p value indicates signficant difference?
pvalue = test[1]
pvalue

0.005185422461637733

In [220]:
dof = test[2]
dof

4

In [221]:
#Was there a time you could not see a doctor because of cost?
cost2014 = df_14['medcost']

In [222]:
cost201518 = df_1518['medcost']

In [223]:
#take random sample same size as 2014 data
sample1518 = cost201518.sample(n = 51858, random_state = 1)

In [224]:
#1 corresponds to not being able to see a doctor and 2 is being able to see a doctor
counts14 = getCounts(cost2014, 2)
counts14

[4177, 47546]

In [225]:
counts1518 = getCounts(sample1518, 2)
counts1518

[4097, 47633]

In [226]:
#total number of nonrefused or not sure answers
total14 = np.sum(counts14)
total1518 = np.sum(counts1518)

print("2014 total:", total14)
print("2015-2018 total:",total1518)

2014 total: 51723
2015-2018 total: 51730


In [227]:
#create combined matrix
cost = pd.DataFrame([counts14, counts1518], index = ["2014", "2015-2018"], 
                         columns = ["Yes", "No"])
cost

,Yes,No
2014,4177,47546
2015-2018,4097,47633


In [228]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2014', x = cost.columns, y = cost.iloc[0]),
                       go.Bar(name = '2015-18', x = cost.columns, y = cost.iloc[1])])
fig.update_layout(barmode='group')
fig.show()

In [229]:
#perform chi2 test
test1 = chi2_contingency(cost)
test1

(0.8315231241843959,
 0.361832471245386,
 1,
 array([[ 4136.72007578, 47586.27992422],
        [ 4137.27992422, 47592.72007578]]))

In [230]:
#find contingency matrix for data
df = test1[3]

ev = pd.DataFrame(data = df[:,:], index = ["2014", "2015-2018"],
                  columns = ["Yes","No"]).round(2)

ev

,Yes,No
2014,4136.72,47586.28
2015-2018,4137.28,47592.72


In [231]:
#print summary statistics
#note only 1 degree of freedom, however p value indicates no significant difference
chi2 = test1[0]
pvalue = test1[1]
dof = test1[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)

chi2: 0.8315231241843959 p-value: 0.361832471245386 dof: 1


In [232]:
#splitting cost data on race/income

lowcost14 = low_income14['medcost']
lowcost1518 = low_income1518['medcost']

sample1518 = lowcost1518.sample(n = 11099, random_state = 1)

#get counts for each value of having care or not in 2014 data set
counts14 = getCounts(lowcost14, 2)
total = np.sum(counts14)
#get counts for each value of having care or not in 2014 data set
counts1518 = getCounts(sample1518, 2)
        
lowcost = pd.DataFrame([counts14, counts1518], index = ["2014", "2015-2018"], 
                         columns = ["Yes", "No"])
lowcost


,Yes,No
2014,1745,9354
2015-2018,1610,9439


In [233]:
fig = go.Figure(data = [go.Bar(name = '2014', x = cost.columns, y = lowcost.iloc[0]),
                       go.Bar(name = '2015-18', x = cost.columns, y = lowcost.iloc[1])])
fig.update_layout(barmode='group')
fig.show()

In [234]:
#perform chi2 test for low income interviewees
test2 = chi2_contingency(lowcost)
chi2 = test2[0]
pvalue = test2[1]
dof = test2[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)

chi2: 5.61462247562772 p-value: 0.017811217717322733 dof: 1


In [235]:
#do you have health care coverage?
care14 = df_14['hlthpln1']
care1518 = df_1518['hlthpln1']
sample1518 = care1518.sample(n = 51858, random_state = 1)

#get counts for each value of having care or not in 2014 data set
columns = [1, 2]
counts14 = [0,0]
for row in care14:
    value = (int)(row)
    if value < 7:
        counts14[value-1] += 1

#get counts for each value of having care or not in 2014 data set
columns = [1, 2]
counts1518 = [0,0]
for row in sample1518:
    value = (int)(row)
    if value < 7:
        counts1518[value-1] += 1
        
care = pd.DataFrame([counts14, counts1518], index = ["2014", "2015-2018"], 
                         columns = ["Yes", "No"])
care

,Yes,No
2014,48933,2739
2015-2018,49145,2540


In [236]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2014', x = care.columns, y = care.iloc[0]),
                       go.Bar(name = '2015-18', x = care.columns, y = care.iloc[1])])
fig.update_layout(barmode='group')
fig.show()

In [237]:
#perform chi2 test
test2 = chi2_contingency(care)
pvalue = test2[1]
dof = test2[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)

chi2: 5.61462247562772 p-value: 0.005002025418579685 dof: 1


In [238]:
#do you have health care coverage (with low income participants)?
#splitting coverage data on race/income

lowcare14 = low_income14['hlthpln1']
lowcare1518 = low_income1518['hlthpln1']

sample1518 = lowcare1518.sample(n = 11111, random_state = 1)

#get counts for each value of having care or not in 2014 data set
counts14 = getCounts(lowcare14, 2)

#get counts for each value of having care or not in 2014 data set
counts1518 = getCounts(sample1518, 2)
        
lowcare = pd.DataFrame([counts14, counts1518], index = ["2014", "2015-2018"], 
                         columns = ["Yes", "No"])
lowcare


,Yes,No
2014,9952,1159
2015-2018,10014,1045


In [239]:
#chart to view data and see if results match up
fig = go.Figure(data = [go.Bar(name = '2014', x = lowcare.columns, y = lowcare.iloc[0]),
                       go.Bar(name = '2015-18', x = lowcare.columns, y = lowcare.iloc[1])])
fig.update_layout(barmode='group')
fig.show()

In [240]:
#perform chi2 test
test2 = chi2_contingency(lowcare)
chi2 = test2[0]
pvalue = test2[1]
dof = test2[2]
print("chi2:", chi2, "p-value:", pvalue, "dof:", dof)

chi2: 5.857989598121653 p-value: 0.015506495786049559 dof: 1


In [241]:
#In general how satisfied are you with the health care you recieved?
hc_sat14 = df_14['carercvd']
hc_sat1518 = df_1518['carercvd']
#sample1518 = hc_sat1518.sample(n = 51858, random_state = 1)
print(hc_sat1518)
#get counts for each value of general health in 2014 data set
columns = [1, 2]
counts14 = [0,0,0]
hc_sat14 = hc_sat14.dropna()
for row in hc_sat14:
    value = (int)(row)
    if value < 7:
        counts14[value-1] += 1

#get counts for each value of general health in 2014 data set
columns = [1, 2]
counts1518 = [0,0,0]
hc_sat1518 = hc_sat1518.dropna()
print(hc_sat1518)
for row in hc_sat1518:
    value = (int)(row)
    if value < 7:
        counts1518[value-1] += 1
        
hc_sat = pd.DataFrame([counts14, counts1518], index = ["2014", "2015-2018"], 
                         columns = ["Very Satisfied", "Somewhat Satisfied", "Not at all Satisfied"])
hc_sat

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
193644   NaN
193645   NaN
193646   NaN
193647   NaN
193648   NaN
Name: carercvd, Length: 193649, dtype: float64
Series([], Name: carercvd, dtype: float64)


,Very Satisfied,Somewhat Satisfied,Not at all Satisfied
2014,29692,10497,1126
2015-2018,0,0,0


In [249]:
# generate table for income1 x hcare for 2014, and for 2015-18

num_samples = 20
summed_cross_14 = np.zeros(shape=(8, 6))
summed_cross_1518 = np.zeros(shape=(8, 6))

for sample_i in range(num_samples): # to average across multiple samples of 2015-2018 data
    sample_df1518 = df_1518.sample(n = 51858)#, random_state = 1)

    income_x_hcare_14 = np.zeros(shape=(8, 6)) #income by hlthpln
    income_x_hcare_1518 = np.zeros(shape=(8, 6)) #income by hlthpln

    hlthpln_options = [1,2,7,9]

    for i in range(1,9): # for each income group (<$10,000, <$15,000, <$20,000, <$25,000, <$35,000, <$50,000, <$75,000, >$75,000)
        for j in range(4): # for each healthplan group (Yes, no, not sure, refused)
            hlthpln_i = hlthpln_options[j]
            income_x_hcare_14[i-1, j] = len(df_14.query('income2 == @i & hlthpln1 == @hlthpln_i')) 
            income_x_hcare_1518[i-1, j] = len(sample_df1518.query('income2 == @i & hlthpln1 == @hlthpln_i'))
        
        # calculate total respondents
        income_x_hcare_14[i-1, 4] = income_x_hcare_14[i-1, 0] + income_x_hcare_14[i-1, 1]
        income_x_hcare_1518[i-1, 4] = income_x_hcare_1518[i-1, 0] + income_x_hcare_1518[i-1, 1]
    
        # calculate % that said Yes
        income_x_hcare_14[i-1, 5] = (income_x_hcare_14[i-1, 0] / income_x_hcare_14[i-1, 4]).round(4) * 100
        income_x_hcare_1518[i-1, 5] = (income_x_hcare_1518[i-1, 0] / income_x_hcare_1518[i-1, 4]).round(4) * 100

    summed_cross_14 += income_x_hcare_14
    summed_cross_1518 += income_x_hcare_1518
    
averaged_cross_14 = summed_cross_14 / num_samples
averaged_cross_1518 = summed_cross_1518 / num_samples
    
income_x_hcare_14_df = pd.DataFrame(income_x_hcare_14, 
                 index = ["<$10,000", "<$15,000", "<$20,000", "<$25,000", "<$35,000", "<$50,000", "<$75,000", ">$75,000"], 
                 columns=["Yes", "No", "Not Sure", "Refused", "Total Respondents", "Yes Percentage"])

income_x_hcare_1518_df = pd.DataFrame(income_x_hcare_1518, 
                 index = ["<$10,000", "<$15,000", "<$20,000", "<$25,000", "<$35,000", "<$50,000", "<$75,000", ">$75,000"], 
                 columns=["Yes", "No", "Not Sure", "Refused", "Total Respondents", "Yes Percentage"])

print("Q: Do you have any health care coverage? (2014)")
income_x_hcare_14_df

Q: Do you have any health care coverage? (2014)


,Yes,No,Not Sure,Refused,Total Respondents,Yes Percentage
"<$10,000",1539.0,224.0,4.0,1.0,1763.0,87.29
"<$15,000",2171.0,224.0,5.0,5.0,2395.0,90.65
"<$20,000",2734.0,330.0,2.0,2.0,3064.0,89.23
"<$25,000",3508.0,381.0,6.0,7.0,3889.0,90.20
"<$35,000",4136.0,342.0,13.0,4.0,4478.0,92.36
"<$50,000",5652.0,373.0,8.0,4.0,6025.0,93.81
"<$75,000",6854.0,229.0,3.0,3.0,7083.0,96.77
">$75,000",14823.0,182.0,12.0,3.0,15005.0,98.79


In [250]:
print("Q: Do you have any health care coverage? (Sampled 2015-2018)")
print("Colored based on change from 2014 -> green = increase, pink = decrease, white = no change")

def highlight_change(current_df, other_df): 
    return pd.DataFrame(np.where(current_df.gt(other_df), 'background-color: lightgreen', 
                                 np.where(current_df.lt(other_df), 'background-color: pink', '')), # else
                        index=current_df.index, 
                        columns=current_df.columns)

income_x_hcare_1518_df.style.apply(highlight_change, other_df=income_x_hcare_14_df, axis=None)

#TODO: Percent with health care coverage seems to have increased, check significance / chi-squared?

Q: Do you have any health care coverage? (Sampled 2015-2018)
Colored based on change from 2014 -> green = increase, pink = decrease, white = no change


,Yes,No,Not Sure,Refused,Total Respondents,Yes Percentage
"<$10,000",1446.000000,157.000000,7.000000,3.000000,1603.000000,90.210000
"<$15,000",1982.000000,160.000000,11.000000,7.000000,2142.000000,92.530000
"<$20,000",2506.000000,281.000000,5.000000,2.000000,2787.000000,89.920000
"<$25,000",2820.000000,297.000000,2.000000,2.000000,3117.000000,90.470000
"<$35,000",3757.000000,270.000000,11.000000,7.000000,4027.000000,93.300000
"<$50,000",5487.000000,299.000000,5.000000,2.000000,5786.000000,94.830000
"<$75,000",6761.000000,213.000000,5.000000,0.000000,6974.000000,96.950000
">$75,000",15716.000000,303.000000,10.000000,2.000000,16019.000000,98.110000


In [ ]:
# generate table for income1 x genhlth for 2014, and for 2015-18

num_samples = 20

summed_cross_14 = np.zeros(shape=(8, 16))
summed_cross_1518 = np.zeros(shape=(8, 16))

for sample_i in range(num_samples): # to average across multiple samples of 2015-2018 data
    sample_df1518 = df_1518.sample(n = 51858)

    income_x_health_14 = np.zeros(shape=(8, 16)) #income by genhlth
    income_x_health_1518 = np.zeros(shape=(8, 16)) #income by genhlth

    genhlth_options = [1,2,3,4,5,7,9]

    for i in range(1,9): # for each income group (<$10,000, <$15,000, <$20,000, <$25,000, <$35,000, <$50,000, <$75,000, >$75,000)
        for j in range(7): # for each genhlth option (1=excellent through 5=poor, with 7=not sure and 9=refused)
            genhlth_i = genhlth_options[j]
            income_x_health_14[i-1, j] = len(df_14.query('income2 == @i & genhlth == @genhlth_i')) 
            income_x_health_1518[i-1, j] = len(sample_df1518.query('income2 == @i & genhlth == @genhlth_i'))
        
        # calculate total respondents
        for opt in range(5):
            income_x_health_14[i-1, 7] += income_x_health_14[i-1, opt]
            income_x_health_1518[i-1, 7] += income_x_health_1518[i-1, opt]
    
        # calculate % that said #
        for opt in range(5):
            income_x_health_14[i-1, 8+opt] = (income_x_health_14[i-1, opt] / income_x_health_14[i-1, 7]).round(4) * 100
            income_x_health_1518[i-1, 8+opt] = (income_x_health_1518[i-1, opt] / income_x_health_1518[i-1, 7]).round(4) * 100

        income_x_health_14[i-1, 13] = income_x_health_14[i-1, 8] + income_x_health_14[i-1, 9] + income_x_health_14[i-1, 10]
        income_x_health_1518[i-1, 13] = income_x_health_1518[i-1, 8] + income_x_health_1518[i-1, 9] + income_x_health_1518[i-1, 10]
    
        income_x_health_14[i-1, 14] = income_x_health_14[i-1, 11] 
        income_x_health_1518[i-1, 14] = income_x_health_1518[i-1, 11]
    
        income_x_health_14[i-1, 15] = income_x_health_14[i-1, 12] 
        income_x_health_1518[i-1, 15] = income_x_health_1518[i-1, 12]
        
    summed_cross_14 += income_x_health_14
    summed_cross_1518 += income_x_health_1518
    
averaged_cross_14 = summed_cross_14 / num_samples
averaged_cross_1518 = summed_cross_1518 / num_samples
    
income_x_health_14_df = pd.DataFrame(averaged_cross_14, 
                 index = ["<$10,000", "<$15,000", "<$20,000", "<$25,000", "<$35,000", "<$50,000", "<$75,000", ">$75,000"], 
                 columns=["Excellent", "Very Good", "Good", "Fair", "Poor", "Not Sure", "Refused", "Total Respondents", "% Excellent", "% Very Good", "% Good", "% Fair", "% Poor", "%Positive (1-3)", "%Neutral", "%Negative (5)"])

income_x_health_1518_df = pd.DataFrame(averaged_cross_1518, 
                 index = ["<$10,000", "<$15,000", "<$20,000", "<$25,000", "<$35,000", "<$50,000", "<$75,000", ">$75,000"], 
                 columns=["Excellent", "Very Good", "Good", "Fair", "Poor", "Not Sure", "Refused", "Total Respondents", "% Excellent", "% Very Good", "% Good", "% Fair", "% Poor", "%Positive (1-3)", "%Neutral", "%Negative (5)"])

print("Q: General health of the participant (1 = excellent -> 5 = poor) (2014)")
income_x_health_14_df

In [ ]:
print("Q: General health of the participant (1 = excellent -> 5 = poor) (Sampled 2015-2018)")
print("Colored based on change from 2014 -> green = increase, pink = decrease, white = no change")
 
income_x_health_1518_df.style.apply(highlight_change, other_df=income_x_health_14_df, axis=None)